In [1]:
!pip install wandb pandas transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 156.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 158.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, random_split
import wandb
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import time
import torch
from torch.utils.data import Dataset

In [4]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from google.colab import files
uploaded = files.upload()

In [5]:
df = pd.read_csv("combined.csv")

In [6]:
df

,Game Number,Move Number,Player,Move,Board,Comment
0,2263,1,White,d2d4,r n b q k b n r\np p p p p p p p\n. . . . . . ...,NaN
1,2263,2,Black,g8f6,r n b q k b n r\np p p p p p p p\n. . . . . . ...,NaN
2,2263,3,White,c2c4,r n b q k b . r\np p p p p p p p\n. . . . . n ...,NaN
3,2263,4,Black,g7g6,r n b q k b . r\np p p p p p p p\n. . . . . n ...,NaN
4,2263,5,White,b1c3,r n b q k b . r\np p p p p p . p\n. . . . . n ...,NaN
...,...,...,...,...,...,...
184924,3,94,Black,h5h3,. . . . . . . .\n. . . . k . p .\n. p n . . p ...,NaN
184925,3,95,White,b1b6,. . . . . . . .\n. . . . k . p .\n. p n . . p ...,There goes the extra pawn.
184926,3,96,Black,e7d6,. . . . . . . .\n. . . . k . p .\n. R n . . p ...,NaN
184927,3,97,White,g1g2,. . . . . . . .\n. . . . . . p .\n. R n k . p ...,NaN


In [7]:
df = df.dropna().reset_index()

In [8]:
df

,index,Game Number,Move Number,Player,Move,Board,Comment
0,22,2263,23,White,e1d2,r n b . . r k .\np p . . p p b p\n. . . . . . ...,(In this line White has to accept some risks. ...
1,24,2263,25,White,d4d5,r n b r . . k .\np p . . p p b p\n. . . . . . ...,(The first step of White's strategical plan: t...
2,25,2263,26,Black,b8c6,r n b r . . k .\np p . . p p b p\n. . . . . . ...,(Taking advantage of the pin.)
3,28,2263,29,White,d5d6,r . b r . . k .\np p . . . p b p\n. . n . p . ...,(The second step.)
4,29,2263,30,Black,b7b6,r . b r . . k .\np p . . . p b p\n. . n P p . ...,(What about 15...e5? A good plan would be then...
...,...,...,...,...,...,...,...
20768,184912,3,82,Black,d6f7,. . . . . . . .\n. . . . k . p .\n. p q n . p ...,Going toward the king-side isn't necessary now...
20769,184915,3,85,White,f3e5,. . . . . . . .\n. . . . k n p .\n. p q . . p ...,a very nifty tactical shot!
20770,184920,3,90,Black,f7g5,. . . . . . . .\n. . . . k n p .\n. p n . . p ...,"Kasparov has won a pawn, but his attack on Kg1..."
20771,184925,3,95,White,b1b6,. . . . . . . .\n. . . . k . p .\n. p n . . p ...,There goes the extra pawn.


In [9]:
import re

def board_to_fen(board, player, move_number):
    # Extract rows and reverse to match FEN order (from 8th to 1st rank)
    rows = board.strip().split('\n')

    # Convert board rows to FEN format
    fen_rows = []
    for row in rows:
        clean_row = re.sub(r"\s+", "", row)  # Remove spaces
        # Correctly replace consecutive dots with the count
        fen_row = re.sub(r"(\.+) ", lambda m: str(len(m.group(1))), clean_row)  # Incorrectly placed space
        fen_row = re.sub(r"\.+", lambda m: str(len(m.group(0))), fen_row)  # Handle all dots
        fen_rows.append(fen_row)

    fen_pieces = "/".join(fen_rows)

    # Active color ('w' for White, 'b' for Black)
    active_color = 'w' if player == 'White' else 'b'

    # Default values for other FEN components (assuming basic capabilities)
    castling_availability = 'KQkq'  # Assuming all castling is still possible
    en_passant_target = '-'  # No en passant target
    halfmove_clock = '0'  # Reset on pawn moves or captures, not shown here
    fullmove_number = str(move_number)

    # Compile full FEN string
    fen = f"{fen_pieces} {active_color} {castling_availability} {en_passant_target} {halfmove_clock} {fullmove_number}"
    return fen

# Assuming df is your DataFrame
df['FEN'] = df.apply(lambda row: board_to_fen(row['Board'], row['Player'], row['Move Number']), axis=1)
df.head()


,index,Game Number,Move Number,Player,Move,Board,Comment,FEN
0,22,2263,23,White,e1d2,r n b . . r k .\np p . . p p b p\n. . . . . . ...,(In this line White has to accept some risks. ...,rnb2rk1/pp2ppbp/6p1/8/3PP3/4BN2/P2q1PPP/1R2KB1...
1,24,2263,25,White,d4d5,r n b r . . k .\np p . . p p b p\n. . . . . . ...,(The first step of White's strategical plan: t...,rnbr2k1/pp2ppbp/6p1/8/3PP3/4BN2/P2K1PPP/1R3B1R...
2,25,2263,26,Black,b8c6,r n b r . . k .\np p . . p p b p\n. . . . . . ...,(Taking advantage of the pin.),rnbr2k1/pp2ppbp/6p1/3P4/4P3/4BN2/P2K1PPP/1R3B1...
3,28,2263,29,White,d5d6,r . b r . . k .\np p . . . p b p\n. . n . p . ...,(The second step.),r1br2k1/pp3pbp/2n1p1p1/2BP4/4P3/5N2/P2K1PPP/1R...
4,29,2263,30,Black,b7b6,r . b r . . k .\np p . . . p b p\n. . n P p . ...,(What about 15...e5? A good plan would be then...,r1br2k1/pp3pbp/2nPp1p1/2B5/4P3/5N2/P2K1PPP/1R3...


In [11]:
config = {
    'Model': 'gpt2',
    'NUMER_OF_DATA_DIRS': 12,
    'batch_size': 2,
    'lr':  3e-5,
    'train_precentege': 0.9,
    'epochs': 100,
    'data_to_use': {'<fen>': True, '<moves>': True, '<last move description>': False,
                    '<legal moves>': False, '<attacked by>': False, '<attacks>': False},
    'max_length':300
}

In [12]:
# combined csv
# def convert_data_to_text(row, max_length=config['max_length'], end_of_text_token=""):
#     # Assuming 'FEN', 'moves', and 'comment' are column names in the DataFrame
#     FEN, moves, comment = row['FEN'], row['Move'], row['Comment']
#     FEN, moves, comment = FEN[:max_length], moves[:max_length], comment[:max_length]
#     token_to_data = {'<fen>': FEN, '<moves>': moves}
#     text = ""
#     for token, data in token_to_data.items():
#         text += f"{token} {data} "
#     text += f"<comment> {comment} {end_of_text_token}"
#     return text

def convert_data_to_text(row, max_length=config['max_length'], end_of_text_token=""):
    # Assuming 'FEN', 'moves', and 'comment' are column names in the DataFrame
    FEN, moves, comment = row['FEN'], row['Move'], row['Comment']
    FEN, moves, comment = FEN[:max_length], moves[:max_length], comment[:max_length]

    # Split moves into a list
    move_list = moves.split(',')

    # Truncate move history to last 5 moves (or less for initial moves)
    history_moves = move_list[-min(len(move_list), 5):]  # Get last 5 or less moves
    history_moves_str = ", ".join(history_moves)  # Join moves with comma separator

    token_to_data = {'<fen>': FEN, '<moves>': history_moves_str}
    text = ""
    for token, data in token_to_data.items():
        text += f"{token} {data} "
    text += f"<comment> {comment} {end_of_text_token}"
    return text


# def convert_data_to_text_2(data_object, max_length=768, end_of_text_token="<|endoftext|>"):
#     (FEN, moves, last_move_desc, legal_moves, attackers_list, attacks_list, comment) = data_object
#     (FEN, moves, last_move_desc, legal_moves,
#      attackers_list, attacks_list, comment) = (FEN[:max_length], moves[:max_length], last_move_desc[:max_length],
#                                                legal_moves[:max_length], attackers_list[:max_length],
#                                                attacks_list[:max_length], comment[:max_length])
#     moves = moves.split(',')[-1].strip() if ',' in moves else moves

#     token_to_data = {'<fen>': FEN, '<moves>': moves, '<last move description>': last_move_desc,
#                      '<legal moves>': legal_moves, '<attacked by>': attackers_list, '<attacks>': attacks_list}
#     text = ""
#     for token in token_to_data.keys():
#         if config['data_to_use'][token]:
#             text += f"{token} {token_to_data[token]} "
#     text += f"<comment> {comment} {end_of_text_token}"  # comment always included at the end + end token

#     return text

def convert_data_to_text_2(data_object, max_length=768, end_of_text_token="<|endoftext|>"):
    (FEN, moves, last_move_desc, legal_moves, attackers_list, attacks_list, comment) = data_object
    (FEN, moves, last_move_desc, legal_moves,
     attackers_list, attacks_list, comment) = (FEN[:max_length], moves[:max_length], last_move_desc[:max_length],
                                               legal_moves[:max_length], attackers_list[:max_length],
                                               attacks_list[:max_length], comment[:max_length])
    moves = moves.split(',')[-1].strip() if ',' in moves else moves

    # Split moves into a list
    move_list = moves.split(',')

    # Truncate move history to last 5 moves (or less for initial moves)
    history_moves = move_list[-min(len(move_list), 5):]  # Get last 5 or less moves
    history_moves_str = ", ".join(history_moves)  # Join moves with comma separator

    token_to_data = {'<fen>': FEN, '<moves>': history_moves_str, '<last move description>': last_move_desc,
                     '<legal moves>': legal_moves, '<attacked by>': attackers_list, '<attacks>': attacks_list}
    text = ""
    for token in token_to_data.keys():
        if config['data_to_use'][token]:
            text += f"{token} {token_to_data[token]} "
    text += f"<comment> {comment} {end_of_text_token}"  # comment always included at the end + end token

    return text



In [13]:
import pickle

dataset_tokens = list(config['data_to_use'].keys()) + ['<comment>']


class ProcessDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=config['max_length']):
        self.tokenizer = tokenizer
        self.comment_encoding = tokenizer.get_added_vocab()['<comment>']
        self.proccessed_data = []
        self.attn_masks = []
        self.labels = []

        for _, row in df.iterrows():
            text = convert_data_to_text(row, max_length)
            enc_text = tokenizer(text, truncation=True, max_length=max_length, padding="max_length")
            inputs = enc_text['input_ids']
            label_idx = inputs.index(self.comment_encoding) + 1
            labels = [-100] * label_idx + inputs[label_idx:]

            self.proccessed_data.append(torch.tensor(inputs))
            self.attn_masks.append(torch.tensor(enc_text['attention_mask']))
            self.labels.append(torch.tensor(labels))

    def __len__(self):
        return len(self.proccessed_data)

    def __getitem__(self, index):
        return self.proccessed_data[index], self.attn_masks[index], self.labels[index]

class MovesDataset(Dataset):
    def __init__(self, paths, tokenizer, max_length=768):

        self.comment_encoding = tokenizer.get_added_vocab()['<comment>']

        self.proccessed_data = []
        self.attn_masks = []
        self.labels = []


        for path in paths:
            with open(path, 'rb') as file:
                raw_data = pickle.load(file)
            for data_object in raw_data:
                text = convert_data_to_text_2(data_object)

                enc_text = tokenizer(text, truncation=True, max_length=max_length, padding="max_length")

                inputs = enc_text['input_ids']
                label_idx = inputs.index(self.comment_encoding) + 1
                labels = [-100] * label_idx + inputs[label_idx:]

                self.proccessed_data.append(torch.tensor(inputs))
                self.attn_masks.append(torch.tensor(enc_text['attention_mask']))
                self.labels.append(torch.tensor(labels))

    def __len__(self):
        return len(self.proccessed_data)

    def __getitem__(self, index):
        return self.proccessed_data[index], self.attn_masks[index], self.labels[index]


In [14]:
import torch
from transformers import GPT2LMHeadModel, GPT2Config, GPT2Tokenizer

GPT2_TYPE = "gpt2"

class GPT2:
    def __init__(self):
        print("Initialization\n")
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.tokenizer = GPT2Tokenizer.from_pretrained(GPT2_TYPE)
        special_tokens_dict = {
            'pad_token': '[PAD]',
            'additional_special_tokens': dataset_tokens
        }
        self.tokenizer.add_special_tokens(special_tokens_dict)
        # self.tokenizer.add_tokens(get_chess_tokens())

        self.configuration = GPT2Config.from_pretrained(GPT2_TYPE)
        self.model = GPT2LMHeadModel(self.configuration)

        # Resize token embeddings to accommodate new tokens
        self.model.resize_token_embeddings(len(self.tokenizer))

        self.model.to(self.device)

    def load_model(self, model_path):
        print("Loading model\n")
        state_dict = torch.load(model_path, map_location=self.device)
        self.model.load_state_dict(state_dict)
        self.model.to(self.device)


In [32]:
gpt2 = GPT2()
model = gpt2.model
model.load_state_dict(torch.load("/workspace/models/final.bin"))
model.train()
tokenizer = gpt2.tokenizer
max_length = config['max_length']
eof = '<|endoftext|>'

Initialization



In [33]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50265, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50265, bias=False)
)

In [34]:
games_data_path = '/workspace'
saved_models_path = '/workspace/models/'

In [18]:
df["Comment"] = df["Comment"].fillna('No Comment')

In [35]:
dataset = ProcessDataset(df,tokenizer)
# dataset = MovesDataset([f'{games_data_path}1.p'], tokenizer, max_length=max_length)


In [36]:
train_size = int(config['train_precentege'] * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)


In [37]:
run = wandb.init(project="LmChess", config={'batch size': config['batch_size'], 'lr': config['lr'], 'epochs': config['epochs']})


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
loss,▂▄▄▁▂▄▅▇▃▂▂▁▂▄▇▃▄█▄▂▅▆▇▂▂▃▄▄▇▂▃▁▃▂▇▁▅▂▃▁
epoch,99
loss,0.61295


In [38]:
#validation

In [39]:
validation_proccessed_data, validation_attn_masks, validation_labels = next(iter(test_dataloader))

validation_input_encodings = []
for i in range(config['batch_size']):
  textual_validation_data = tokenizer.decode(token_ids = validation_proccessed_data[i], skip_special_tokens=False).split('<comment>')

  validation_target_text = textual_validation_data[1].split(eof)[0]
  validation_input_text = textual_validation_data[0]

  wandb.log({f"validation_target_text {i}": wandb.Html(f'<p>{validation_target_text}</p>')})
  wandb.log({f"validation_input_text {i}": wandb.Html(f'<p>{validation_input_text}</p>')})

  comment_idx = list(validation_proccessed_data[i]).index(dataset.comment_encoding) + 1
  validation_input_encoding = validation_proccessed_data[i][:comment_idx].unsqueeze(0).cuda()
  #validation_input_encoding  = tokenizer.encode(validation_input_text, return_tensors="pt").cuda()

  validation_input_encodings.append(validation_input_encoding)

In [40]:
len(train_dataloader)

9348

In [ ]:

optimizer = AdamW(model.parameters(), lr= config['lr'])
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=5000, num_training_steps=-1
)

loss = 0
pad_token_id = tokenizer('[PAD]')['input_ids'][0]

epochs = config['epochs']

for epoch in range(epochs):
    with tqdm(total=len(dataset) / 2) as pbar:
        for idx,entry in enumerate(train_dataloader):

            if idx % 500 == 0 and idx != 0:
              for i in range(config['batch_size']):
                with torch.no_grad():
                    outputs = model.generate(validation_input_encodings[i], num_beams=2, no_repeat_ngram_size=2, max_length=max_length+1, pad_token_id=pad_token_id)
                    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                wandb.log({f"output_text {i}": wandb.Html(f'<p>{output_text}</p>')})

            # if idx % 9000 == 0:
            #   torch.save(model.state_dict(), f'{saved_models_path}{idx}_{time.time()}_{int(loss)}.bin')

            model.zero_grad()

            inputs = entry[0].cuda()
            attn_masks = entry[1].cuda()
            labels = entry[2].cuda()
            outputs = model(inputs, labels=labels, attention_mask = attn_masks)

            loss = outputs['loss']
            loss.backward()
            optimizer.step()
            scheduler.step()

            wandb.log({"epoch": epoch, "loss": loss})
            pbar.update(2)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
18696it [08:14, 37.82it/s]                             
18696it [08:19, 37.44it/s]                             
18696it [08:13, 37.85it/s]                             
18696it [08:13, 37.89it/s]                             
18696it [08:07, 38.32it/s]                             
18696it [08:18, 37.51it/s]                             
18696it [08:07, 38.32it/s]                             
18696it [08:13, 37.89it/s]                             
18696it [08:00, 38.90it/s]                             
18696it [07:50, 39.73it/s]                             
18696it [08:16, 37.68it/s]                             
18696it [08:12, 37.93it/s]                             
18696it [08:22, 37.

In [ ]:
torch.save(model.state_dict(), f'{saved_models_path}final100.bin')

test

In [27]:
model_path = '/workspace/models/final100.bin'

In [28]:
gpt2_test = GPT2()

gpt2_test.load_model(model_path)

gpt2_test.model = gpt2_test.model.eval().cuda()


Initialization

Loading model



In [29]:
tested_model = gpt2_test
max = config['max_length']
eof = '<|endoftext|>'
pad_token_id = tested_model.tokenizer('[PAD]')['input_ids'][0]

In [30]:
dataloader = test_dataloader
def get_results():
    # Set device based on availability of CUDA
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move the model to the appropriate device
    tested_model.model.to(device)

    # Retrieve batch data
    processed_data, attn_masks, labels = next(iter(dataloader))

    # Initialize input encodings container
    input_encodings = []

    for i in range(config['batch_size']):
        # Decode the token_ids to text and split at <comment>
        textual_data = tested_model.tokenizer.decode(token_ids=processed_data[i], skip_special_tokens=False).split('<comment>')

        # Target text ends at EOF (assuming EOF is defined elsewhere in your code)
        target_text = textual_data[1].split(eof)[0]
        input_text = textual_data[0]

        # Find the index of <comment> and prepare input encoding
        comment_idx = list(processed_data[i]).index(tested_model.tokenizer.get_added_vocab()['<comment>']) + 1
        input_encoding = processed_data[i][:comment_idx].unsqueeze(0).to(device)

        input_encodings.append(input_encoding)

    # Initialize results container
    results = []

    for input_encoding in input_encodings:
        with torch.no_grad():
            # Generate output using the model's generate function
            outputs = tested_model.model.generate(
                input_encoding,
                num_beams=2,
                no_repeat_ngram_size=2,
                max_length=max + 1,  # assuming 'max' is defined as the maximum length
                pad_token_id=pad_token_id  # assuming 'pad_token_id' is defined elsewhere
            )
            output_text = tested_model.tokenizer.decode(outputs[0], skip_special_tokens=True)
            results.append(output_text)

    return results


In [31]:
results = get_results()
results

[" rnbqkb1r/pppppp1p/5np1/8/3P4/5N2/PPP1PPPP/RNBQKB1R w KQkq - 0 5  c2c4  This is a pawns.  White's pieces. The Black's king, but the pawn. He's pawn at d4, and the f-pawn. Now the d3-file, he's without the Black has to play the e4-f4 and Black to the Bishop. White to be to Black Queen, the Queen. But the White without pawn, it's Bishop, so the Knight, Black without without Black, for the center, as the without e5-Knight,, which without to e3, without f4. Black pawn is to win, to f7-c7,\n awn is not be a-h4 is the c4 (   Black King, with his pieces on the King to his Queen's Queen and without d5, White pawn to attack. This's d-d4 to a without his pawn on Black Knight's position, on d7. I without White, in the a7 and f5. It's King's attack the\nthe pawn and e7 to without it without\ns, at f1-e5 and d2-square, a Knight. In the bishop.Nf5  without",
 " r2qr1k1/1ppb1pp1/p1np1bnp/4p3/1P1PP3/PNP1BN2/2B2PPP/R2Q1RK1 b KQkq - 0 26  f6g5  Black's pieces on the pawns, but the f-pawn. awn is a paw

evaluation

In [30]:
!pip install datasets tensorflow nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 15.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 134.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 96.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 92.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [31]:
from tqdm import tqdm
import torch
from datasets import load_metric
import tensorflow as tf
from nltk.translate.bleu_score import sentence_bleu


def perplexity(model, dataloader):
    eval_loss = 0
    with tqdm(total=len(dataloader)) as pbar:
        for idx, entry in enumerate(dataloader):
            with torch.no_grad():
                inputs = entry[0].cuda()
                attn_masks = entry[1].cuda()
                labels = entry[2].cuda()
                outputs = model(inputs, labels=labels, attention_mask=attn_masks)
            loss = outputs[0]
            eval_loss += loss.mean().item()
            pbar.update(2)
    final_eval_loss = eval_loss / len(dataloader)
    perplexity = torch.exp(torch.tensor(final_eval_loss))
    return perplexity


def bleurt(target_texts, output_texts):
    metric = load_metric("bleurt")
    tf.compat.v1.flags.DEFINE_string('f', '', '')

    scores = metric.compute(predictions=output_texts, references=target_texts)['scores']
    return scores


def bleu(target_texts, output_texts):
    scores = []
    for idx in range(len(output_texts)):
        reference = [target_texts[idx].split()]
        candidate = output_texts[idx].split()
        scores.append(sentence_bleu(reference, candidate))
    return scores

2024-04-24 02:33:49.383598: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 02:33:50.479411: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [32]:
test_perplexity = perplexity(tested_model.model, dataloader)
print(test_perplexity)

914it [00:04, 185.91it/s]                         

tensor(1.8497)


In [33]:
import torch
from tqdm import tqdm

def get_targets_and_outputs(model, dataset, comment_encoding, pad_token_id, max_length=768, eof='<|endoftext|>'):
    target_texts = []
    output_texts = []
    with tqdm(total=len(dataset)) as pbar:
        for idx, entry in enumerate(dataset):

          textual_data = model.tokenizer.decode(token_ids=entry[0], skip_special_tokens=False)
          textual_data = textual_data.split('<comment>')[1].split(eof)[0]
          target_texts.append(textual_data)

          comment_idx = list(entry[0]).index(comment_encoding) + 1
          input_encoding = entry[0][:comment_idx].unsqueeze(0).cuda()
          with torch.no_grad():
              outputs = model.model.generate(input_encoding, num_beams=2, no_repeat_ngram_size=2, max_length=max_length+1, pad_token_id=pad_token_id)
              output_text = model.tokenizer.decode(outputs[0], skip_special_tokens=False)
              output_text = output_text.split('<comment>')[1].split(eof)[0]
          output_texts.append(output_text)

          pbar.update(1)
    return target_texts, output_texts

In [34]:
target_texts, output_texts = get_targets_and_outputs(tested_model, dataset, dataset.comment_encoding, pad_token_id, max_length=max, eof=eof)

100%|██████████| 9129/9129 [5:57:02<00:00,  2.35s/it]  


In [4]:
!pip install git+https://github.com/google-research/bleurt.git

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-029f8586
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-029f8586
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 8.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 68.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 77.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 19.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
test_bleurt = bleurt(target_texts, output_texts)

NameError: name 'bleurt' is not defined

In [36]:
print(sum(test_bleurt)/len(test_bleurt))

NameError: name 'test_bleurt' is not defined

In [ ]:
test_bleu = bleu(target_texts, output_texts)

In [ ]:
print(sum(test_bleu)/len(test_bleu))